In [1]:
import pymysql

connection = pymysql.connect(
  host = "localhost",
  port = 3306,
  user = "root",
  password = "", 
)
cursor = connection.cursor()

In [2]:
#Creating Database

cursor.execute("create database nutrition")

1

In [4]:
import pandas as pd
import pymysql


# MySQL connection to XAMPP's phpMyAdmin MySQL
connection = pymysql.connect(
    host="localhost",
    port=3306,
    user="root",
    password="",
    database="nutrition"
)


# SQL to create tables
sql_create_product_info = """
CREATE TABLE IF NOT EXISTS product_info (
    product_code VARCHAR(64) PRIMARY KEY,
    product_name TEXT,
    brand TEXT
)
"""

sql_create_nutrient_info = """
CREATE TABLE IF NOT EXISTS nutrient_info (
    product_code VARCHAR(64),
    energy_kcal_value FLOAT,
    energy_kj_value FLOAT,
    carbohydrates_value FLOAT,
    sugars_value FLOAT,
    fat_value FLOAT,
    saturated_fat_value FLOAT,
    proteins_value FLOAT,
    fiber_value FLOAT,
    salt_value FLOAT,
    sodium_value FLOAT,
    nova_group INT,
    nutrition_score_fr INT,
    fruits_vegetables_nuts_estimate_from_ingredients_100g FLOAT,
    FOREIGN KEY (product_code) REFERENCES product_info(product_code)
)
"""

sql_create_derived_metrics = """
CREATE TABLE IF NOT EXISTS derived_metrics (
    product_code VARCHAR(64),
    sugar_to_carb_ratio FLOAT,
    calorie_category TEXT,
    sugar_category TEXT,
    is_ultra_processed TEXT,
    FOREIGN KEY (product_code) REFERENCES product_info(product_code)
)
"""


def create_tables(conn):
    with conn.cursor() as cursor:
        cursor.execute(sql_create_product_info)
        cursor.execute(sql_create_nutrient_info)
        cursor.execute(sql_create_derived_metrics)
    conn.commit()


def harmonize_product_codes(df_list):
    for df in df_list:
        df.loc[:, 'product_code'] = df['product_code'].astype(str).str.strip().str.lower()


def print_unmatched_codes(df_product, df_nutrient, df_derived):
    valid_codes = set(df_product['product_code'])
    missing_in_nutrient = set(df_nutrient['product_code']) - valid_codes
    missing_in_derived = set(df_derived['product_code']) - valid_codes

    if missing_in_nutrient:
        print(f"Unmatched product_codes in nutrient_info: {missing_in_nutrient}")
    else:
        print("All product_codes in nutrient_info matched.")

    if missing_in_derived:
        print(f"Unmatched product_codes in derived_metrics: {missing_in_derived}")
    else:
        print("All product_codes in derived_metrics matched.")


def filter_child_data(df_product, df_nutrient, df_derived):
    valid_codes = set(df_product['product_code'])
    df_nutrient_filtered = df_nutrient[df_nutrient['product_code'].isin(valid_codes)]
    df_derived_filtered = df_derived[df_derived['product_code'].isin(valid_codes)]
    return df_nutrient_filtered, df_derived_filtered


def insert_product_info(conn, df_product):
    with conn.cursor() as cursor:
        product_data = [(row['product_code'], row['product_name'], row['brand']) for _, row in df_product.iterrows()]
        cursor.executemany(
            """
            INSERT INTO product_info (product_code, product_name, brand)
            VALUES (%s, %s, %s)
            ON DUPLICATE KEY UPDATE
                product_name=VALUES(product_name),
                brand=VALUES(brand)
            """,
            product_data
        )
    conn.commit()


def insert_nutrient_info(conn, df_nutrient):
    with conn.cursor() as cursor:
        nutrient_data = [
            (
                row['product_code'], row['energy_kcal_value'], row['energy_kj_value'], row['carbohydrates_value'],
                row['sugars_value'], row['fat_value'], row['saturated_fat_value'], row['proteins_value'],
                row['fiber_value'], row['salt_value'], row['sodium_value'], row['nova_group'],
                row['nutrition_score_fr'], row['fruits_vegetables_nuts_estimate_from_ingredients_100g']
            )
            for _, row in df_nutrient.iterrows()
        ]
        cursor.executemany(
            """
            INSERT INTO nutrient_info (
                product_code, energy_kcal_value, energy_kj_value, carbohydrates_value, sugars_value,
                fat_value, saturated_fat_value, proteins_value, fiber_value, salt_value,
                sodium_value, nova_group, nutrition_score_fr, fruits_vegetables_nuts_estimate_from_ingredients_100g
            )
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            ON DUPLICATE KEY UPDATE
                energy_kcal_value=VALUES(energy_kcal_value),
                energy_kj_value=VALUES(energy_kj_value),
                carbohydrates_value=VALUES(carbohydrates_value),
                sugars_value=VALUES(sugars_value),
                fat_value=VALUES(fat_value),
                saturated_fat_value=VALUES(saturated_fat_value),
                proteins_value=VALUES(proteins_value),
                fiber_value=VALUES(fiber_value),
                salt_value=VALUES(salt_value),
                sodium_value=VALUES(sodium_value),
                nova_group=VALUES(nova_group),
                nutrition_score_fr=VALUES(nutrition_score_fr),
                fruits_vegetables_nuts_estimate_from_ingredients_100g=VALUES(fruits_vegetables_nuts_estimate_from_ingredients_100g)
            """,
            nutrient_data
        )
    conn.commit()


def insert_derived_metrics(conn, df_derived):
    with conn.cursor() as cursor:
        derived_data = [
            (
                row['product_code'], row['sugar_to_carb_ratio'], row['calorie_category'],
                row['sugar_category'], row['is_ultra_processed']
            )
            for _, row in df_derived.iterrows()
        ]
        cursor.executemany(
            """
            INSERT INTO derived_metrics (
                product_code, sugar_to_carb_ratio, calorie_category, sugar_category, is_ultra_processed
            )
            VALUES (%s,%s,%s,%s,%s)
            ON DUPLICATE KEY UPDATE
                sugar_to_carb_ratio=VALUES(sugar_to_carb_ratio),
                calorie_category=VALUES(calorie_category),
                sugar_category=VALUES(sugar_category),
                is_ultra_processed=VALUES(is_ultra_processed)
            """,
            derived_data
        )
    conn.commit()


# Load CSV and create DataFrames
csv_path = "Choco_Analysis2.csv"  # Update the path to your CSV file
df = pd.read_csv(csv_path)


# Extract relevant columns from the CSV
df_product = df[['product_code', 'product_name', 'brand']]
df_nutrient = df[['product_code', 'energy_kcal_value', 'energy_kj_value', 'carbohydrates_value', 'sugars_value', 'fat_value', 'saturated_fat_value', 'proteins_value', 'fiber_value', 'salt_value', 'sodium_value', 'nova_group', 'nutrition_score_fr', 'fruits_vegetables_nuts_estimate_from_ingredients_100g']]
df_derived = df[['product_code', 'sugar_to_carb_ratio', 'calorie_category', 'sugar_category', 'is_ultra_processed']]


# Harmonize product_code formatting safely
harmonize_product_codes([df_product, df_nutrient, df_derived])


# Remove rows with missing critical data before database insertion
df_product = df_product.dropna(subset=['product_code', 'product_name', 'brand'])
df_nutrient = df_nutrient.dropna(subset=['product_code'])
df_derived = df_derived.dropna(subset=['product_code'])


# Check for unmatched product_codes
print_unmatched_codes(df_product, df_nutrient, df_derived)


# Create tables if not exist
create_tables(connection)


# Insert product_info data first
insert_product_info(connection, df_product)


# Filter nutrient and derived data to include only known product codes
df_nutrient_filtered, df_derived_filtered = filter_child_data(df_product, df_nutrient, df_derived)


# Insert filtered nutrient_info and derived_metrics
insert_nutrient_info(connection, df_nutrient_filtered)
insert_derived_metrics(connection, df_derived_filtered)

# Close connection
connection.close()

print("Data inserted successfully.")


C:\Users\Dell\AppData\Local\Temp\ipykernel_19160\2414119656.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['6111035000430' '6111242100992' '6111035002175' ... '3229820789910'
 '3564700562802' '7613034975386']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'product_code'] = df['product_code'].astype(str).str.strip().str.lower()
C:\Users\Dell\AppData\Local\Temp\ipykernel_19160\2414119656.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['6111035000430' '6111242100992' '6111035002175' ... '3229820789910'
 '3564700562802' '7613034975386']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'product_code'] = df['product_code'].astype(str).str.strip().str.lower()
C:\Users\Dell\AppData\Local\Temp\ipykernel_19160\2414119656.py:66: FutureWarni

Unmatched product_codes in nutrient_info: {'16103403', '6111265980229', '7622201768836', '6111260232095', '3701002405110', '8434164562883'}
Unmatched product_codes in derived_metrics: {'16103403', '6111265980229', '7622201768836', '6111260232095', '3701002405110', '8434164562883'}
Data inserted successfully.
